# Behind the pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [9]:
!pip install datasets evaluate transformers[sentencepiece]

In [13]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "Both options work for me.",
        "I'm not sure if whether I should hate it or love it.",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9986410737037659},
 {'label': 'NEGATIVE', 'score': 0.9993395209312439}]

In [11]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [25]:
raw_inputs = [
    "The film was shot in Spain.",
    "The film was shot in Romania.",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 1996, 2143, 2001, 2915, 1999, 3577, 1012,  102],
        [ 101, 1996, 2143, 2001, 2915, 1999, 6339, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [16]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [19]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

print(outputs.last_hidden_state [0])

torch.Size([2, 17, 768])
tensor([[ 0.5534,  0.2629,  0.2661,  ...,  0.0838,  0.8821, -0.6527],
        [ 0.1243,  0.2868,  0.5578,  ..., -0.0242,  0.6752, -0.8150],
        [ 0.1289,  0.3760,  0.3716,  ..., -0.1427,  0.5876, -0.5237],
        ...,
        [ 0.6265,  0.3039,  0.4046,  ...,  0.1034,  0.6348, -0.6819],
        [ 0.5774,  0.3270,  0.2795,  ...,  0.1130,  0.7630, -0.6808],
        [ 0.6549,  0.2822,  0.2921,  ...,  0.1357,  0.8096, -0.6796]],
       grad_fn=<SelectBackward0>)


In [26]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [27]:
print(outputs.logits)

tensor([[ 0.4469, -0.3810],
        [-0.6130,  0.6547]], grad_fn=<AddmmBackward0>)


In [23]:
print(outputs.logits.shape)

torch.Size([2, 2])


In [28]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.6959, 0.3041],
        [0.2196, 0.7804]], grad_fn=<SoftmaxBackward0>)


In [29]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}